## Load Base Stable Diffusion Model

In [ ]:
from diffusers import StableDiffusionPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig, SmasherConfig
from pruna.smash import smash
import torch

from diffusers import StableDiffusionPipeline
import torch

model_path = "sayakpaul/sd-model-finetuned-lora-t4"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, use_safetensors=True)
pipe.unet.load_attn_procs(model_path)
# pipe.unet.set_default_attn_processor()
pipe.to("cuda")

prompt = "A pokemon with blue eyes."

## Smash it!

In [ ]:
smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 512
smasher_config['image_width'] = 512
smasher_config['version'] = '1.5'

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="LAION256",
        api_key='your-api-key',
        model_config=None,
        smasher_config=AutoSmasherConfig.from_pretrained("runwayml/stable-diffusion-v1-5"),
        device='cuda',
    )

## Base Model Generation

In [ ]:
%%time
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0].show()

## Smashed Model Generation

In [ ]:
%%time
smashed_model(prompt, num_inference_steps=30)[0]